#Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from scipy.optimize import curve_fit

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
import warnings
warnings.simplefilter("ignore")

#Data

(V, Q, Q_r, S_i, dS, TS_i, dTS, N_i, dN, M, DO, i, Sig/TS, Sig, TKN, X, Th, Th_c, Th_n, Th_cn)

In [ ]:
df = pd.read_csv('/content/ASP_Data.csv')

df.head(3)

,V,Si,So,TSi,TSo,Ni,No,TKNi,TKNo,M,...,Sig,Q,Th_c,Th,dS,dTS,dN,TKN,Thn,Th_cn
0,3132,261,44,309,19,49,11,87,22,2634,...,192,1746.83,9.69,9.17,217,290,38,65,2.2925,11.217412
1,3105,220,37,271,22,44,11,88,23,2649,...,166,1561.05,10.50,9.21,183,249,33,65,2.3025,12.948892
2,2916,207,33,235,21,42,10,79,20,2543,...,140,1481.19,13.41,9.16,174,214,32,59,2.2900,16.777261


In [ ]:
psi_1 = 0.8
psi_2 = 0.2

#C-BOD Removal Regression 1

In [ ]:
df1 = pd.DataFrame()

df1['y'] = df['V']*df['M']/(2*df['Q']*df['dS']*df['dTS'])
df1['x1'] = 2*df['Q']/(df['Si']*df['Sig']*df['V'])
df1['x2'] = df['i']/df['Sig']
df1['x3'] = 1/df['Sig']

df1.head(3)

,y,x1,x2,x3
0,0.037523,0.000022,0.302083,0.005208
1,0.057816,0.000028,0.265060,0.006024
2,0.067225,0.000035,0.335714,0.007143


In [ ]:
y1 = df1['y']
x1 = df1.drop(['y'], axis = 1)

In [ ]:
df1.corr()

,y,x1,x2,x3
y,1.000000,0.856291,0.109267,0.889229
x1,0.856291,1.000000,0.170563,0.941389
x2,0.109267,0.170563,1.000000,0.306288
x3,0.889229,0.941389,0.306288,1.000000


In [ ]:
LR1 = LinearRegression(fit_intercept = False)
LR1.fit(x1, y1)

LinearRegression(fit_intercept=False)

###Equation

In [ ]:
coeff1 = LR1.coef_

c1 = coeff1[0]
c2 = coeff1[1]
c3 = coeff1[2]

print(f'y = ({c1:.2e}).x1 + ({c2:.2e}).x2 + ({c3:.2e}).x3')

y = (3.58e+02).x1 + (-8.93e-02).x2 + (1.09e+01).x3


In [ ]:
y1_pred = LR1.predict(x1)
r21 = r2_score(y1, y1_pred)
r21

0.8049201398308097

###Constants

In [ ]:
Kiu = c3/c2
print(f'Kiu = {Kiu:.1f}')

Kiu = -121.9


In [ ]:
k = 1/(psi_1*c3)
print(f'k = {k:.4f}')

k = 0.1148


In [ ]:
# Km = psi_1*c1*Kd/c3

#C-BOD Removal Regression 2

In [ ]:
df2 = pd.DataFrame()

df2['y'] = 1/df['Th_c']
df2['x1'] = c1/(c3*df['Si'])
df2['x2'] = df['V']/(2*df['Q'])
df2['x3'] = 1 + c2*df['i']/c3

df2.head(3)

,y,x1,x2,x3
0,0.103199,0.125827,0.896481,0.524373
1,0.095238,0.149276,0.994523,0.639180
2,0.074571,0.158651,0.984344,0.614578


In [ ]:
y2 = df2['y']
x2 = df2.drop(['y'], axis = 1)

In [ ]:
df2.corr()

,y,x1,x2,x3
y,1.000000,-0.595637,0.001462,-0.454480
x1,-0.595637,1.000000,-0.015275,0.786238
x2,0.001462,-0.015275,1.000000,0.008093
x3,-0.454480,0.786238,0.008093,1.000000


In [ ]:
def func2(vars, alpha, beta):
  x1, x2, x3 = vars
  return alpha/(x1/x2 + x3) - beta

###Equation

In [ ]:
ind_vars2 = np.vstack((df2['x1'], df2['x2'], df2['x3']))
popt2, pcov2 = curve_fit(func2, ind_vars2, df2['y'])

alpha = popt2[0]
beta = popt2[1]

In [ ]:
y2_pred = func2(ind_vars2, *popt2)
r22 = r2_score(y2, y2_pred)
r22

0.2510385332363331

###Constants

In [ ]:
Kd = beta
# psi_1 = 0.8
# Kd = gamma/psi_1
print(f'Kd = {Kd:.2e}')

Kd = 4.08e-03


In [ ]:
Mu_max = alpha
print(f'Mu_max = {Mu_max:.4f}')

Mu_max = 0.0787


In [ ]:
Km = psi_1*c1*Kd/c3
print(f'Km = {Km:.2e}')

Km = 1.07e-01


###Psi_1 Calc

In [ ]:
df['psi_1'] = np.sqrt(np.abs(2*Km*df['Q']/(Kd*df['Si']*df['V']*(Mu_max*df['Th_c']*(1 - Kd*df['Th_c']) - 1 - df['i']/Kiu))))

df['psi_1'] = df['psi_1'].clip(lower = 0, upper = 1)
df.head(3)

,V,Si,So,TSi,TSo,Ni,No,TKNi,TKNo,M,...,Q,Th_c,Th,dS,dTS,dN,TKN,Thn,Th_cn,psi_1
0,3132,261,44,309,19,49,11,87,22,2634,...,1746.83,9.69,9.17,217,290,38,65,2.2925,11.217412,0.733843
1,3105,220,37,271,22,44,11,88,23,2649,...,1561.05,10.50,9.21,183,249,33,65,2.3025,12.948892,0.888140
2,2916,207,33,235,21,42,10,79,20,2543,...,1481.19,13.41,9.16,174,214,32,59,2.2900,16.777261,0.579735


#NH4-N Removal Regression 1

In [ ]:
df3 = pd.DataFrame()

df3['y'] = df['V']*df['M']/(2*df['Q']*df['dN']*df['dTS'])
df3['x1'] = 2*df['Q']/(df['Ni']*df['Sig']*df['V'])
df3['x2'] = df['i']/df['Sig']
df3['x3'] = 1/df['Sig']
df3['x4'] = 1/df['DO']

df3.head(3)

,y,x1,x2,x3,x4
0,0.214277,0.000119,0.302083,0.005208,0.393701
1,0.320615,0.000138,0.265060,0.006024,0.492611
2,0.365535,0.000173,0.335714,0.007143,0.487805


In [ ]:
y3 = df3['y']
x3 = df3.drop(['y'], axis = 1)

In [ ]:
df3.corr()

,y,x1,x2,x3,x4
y,1.000000,0.865928,0.136664,0.818506,0.665093
x1,0.865928,1.000000,0.192996,0.882377,0.683446
x2,0.136664,0.192996,1.000000,0.306288,-0.418113
x3,0.818506,0.882377,0.306288,1.000000,0.700760
x4,0.665093,0.683446,-0.418113,0.700760,1.000000


In [ ]:
def func3(vars, a, b, c, d):
  x1, x2, x3, x4 = vars
  return (a*x1 + b*x2 + c*x3)*(d*x4 - 1)

In [ ]:
ind_vars3 = np.vstack((df3['x1'], df3['x2'], df3['x3'], df3['x4']))

popt3, pcov3 = curve_fit(func3, ind_vars3, df3['y'])

a3 = popt3[0]
b3 = popt3[1]
c3 = popt3[2]
d3 = popt3[3]

###Equation

In [ ]:
print(f'y = (({a3:.2e}).x1 + ({b3:.2e})*x2 + ({c3:.2e})*x3)*(({d3:.2e})*x4 - 1)')

y = ((-1.13e+03).x1 + (7.07e-02)*x2 + (-1.66e+01)*x3)*((-2.93e-01)*x4 - 1)


In [ ]:
y3_pred = func3(ind_vars3, *popt3)
r23 = r2_score(y3, y3_pred)
r23

0.7678669113078319

###Constants

In [ ]:
Ko = d3
print(f'Ko = {Ko:.4f}')

Ko = -0.2930


In [ ]:
Kniu = c3/b3
print(f'Kniu = {Kniu:.1f}')

Kniu = -234.7


In [ ]:
psi_2 = 0.2
kn = 1/(psi_2*c3)
print(f'kn = {kn:.4f}')

kn = -0.3011


#NH4-N Removal Regression 2

In [ ]:
df4 = pd.DataFrame()

df4['y'] = 2*df['Q']/((df['Ni'])*df['V'])
df4['x1'] = (df['Th_cn']**2)*(1 + Ko/df['DO'])
df4['x2'] = df['Th_cn']*(1 + Ko/df['DO'])
df4['x3'] = (1 + df['i']/Kniu)

df4.head(3)

,y,x1,x2,x3
0,0.022765,111.313515,9.923279,0.752913
1,0.022852,143.469707,11.079689,0.812554
2,0.024188,241.241093,14.379051,0.799774


In [ ]:
y = df4['y']
x = df4.drop(['y'], axis = 1)

In [ ]:
df4.corr()

,y,x1,x2,x3
y,1.000000,0.120677,0.097316,0.511465
x1,0.120677,1.000000,0.992962,0.402641
x2,0.097316,0.992962,1.000000,0.369188
x3,0.511465,0.402641,0.369188,1.000000


In [ ]:
def func4(vars, alpha, beta, gamma):
  x1, x2, x3 = vars
  return alpha*x1 + beta*x2 + gamma*x3

In [ ]:
ind_vars4 = np.vstack((df4['x1'], df4['x2'], df4['x3']))

popt4, pcov4 = curve_fit(func4, ind_vars4, df4['y'])

alpha4 = popt4[0]
beta4 = popt4[1]
gamma4 = popt4[2]

###Equation

In [ ]:
print(f'y = ({alpha4:.2e}).x1 + ({beta4:.2e}).x2 + ({gamma4:.2e}).x3')

y = (6.90e-05).x1 + (-1.98e-03).x2 + (4.30e-02).x3


In [ ]:
y4_pred = func4(ind_vars4, *popt4)
r24 = r2_score(y, y4_pred)
r24

0.24718983687240392

###Constants

In [ ]:
Kdn = -alpha4/beta4
print(f'Kdn = {Kdn:.2e}')

Kdn = 3.48e-02


In [ ]:
k_n = -beta4/gamma4
k_n

0.046170874958368836

In [ ]:
Mu_nm = beta4/gamma4
print(f'Mu_nm = {Mu_nm:.2e}')

Mu_nm = -4.62e-02


In [ ]:
psi_2 = 0.2
Knm = -Kdn*psi_2/gamma4
print(f'Knm = {Knm:.4f}')

Knm = -0.1620


###Psi_2 Calc

In [ ]:
df['psi_2'] = 2*Knm*df['Q']/(Kdn*df['Ni']*df['V']*(Mu_nm*(1 + Ko/df['DO'])*df['Th_cn']*(Kdn*df['Th_cn'] - 1) - df['i']/Kniu - 1))

df.head(3)

,V,Si,So,TSi,TSo,Ni,No,TKNi,TKNo,M,...,Th_c,Th,dS,dTS,dN,TKN,Thn,Th_cn,psi_1,psi_2
0,3132,261,44,309,19,49,11,87,22,2634,...,9.69,9.17,217,290,38,65,2.2925,11.217412,0.733843,0.223724
1,3105,220,37,271,22,44,11,88,23,2649,...,10.50,9.21,183,249,33,65,2.3025,12.948892,0.888140,0.200115
2,2916,207,33,235,21,42,10,79,20,2543,...,13.41,9.16,174,214,32,59,2.2900,16.777261,0.579735,0.215064


#Results

####Psi_1

In [ ]:
psi_1 = df['psi_1'].mean()
psi_1_min = df['psi_1'].min()
psi_1_max = df['psi_1'].max()

print(f'psi_1 :\n Range       : ({psi_1_min:.3f}, {psi_1_max:.3f})')
print(f' Mean Value  : {psi_1:.4f}')

psi_1 :
 Range       : (0.528, 1.000)
 Mean Value  : 0.8552


####Psi_2

In [ ]:
psi_2 = df['psi_2'].mean()
psi_2_min = df['psi_2'].min()
psi_2_max = df['psi_2'].max()

print(f'psi_2 :\n Range       : ({psi_2_min:.4f}, {psi_2_max:.4f})')
print(f' Mean Value  : {psi_2:.4f}')

psi_2 :
 Range       : (0.1544, 0.2713)
 Mean Value  : 0.1999


####Constants

In [ ]:
print(f'Mu_max = {Mu_max:.1f}')
print(f'Mu_nm = {Mu_nm:.2f}\n')
print(f'Kd = {Kd:.3f}')
print(f'Kdn = {Kdn:.3f}\n')
print(f'Km = {Km:.3f}')
print(f'Knm = {Knm:.4f}')

Mu_max = 0.1
Mu_nm = -0.05

Kd = 0.004
Kdn = 0.035

Km = 0.107
Knm = -0.1620
